<a href="https://colab.research.google.com/github/Rekt77/kisa_insuretech/blob/master/torch_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.init
import glob

# gpu사용
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(1000)
if device == "cuda":
  torch.cuda.manual_seed(1000)

learning_rate=0.001
epochs=15

In [59]:
# MNIST dataset
mnist_train = datasets.MNIST(root='/content/drive/My Drive/data/MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = datasets.MNIST(root='/content/drive/My Drive/data/MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [60]:
batch_size=100
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [11]:
data_loader

In [12]:
mnist_single_batch = next(iter(data_loader))

In [15]:
img_array = np.array(mnist_single_batch[0]*255).squeeze(1)
img_array[0] # 각각의 데이터

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,  64., 128., 191., 255., 191.,  64.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0

In [19]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from PIL import Image

for i in range(10):
  mnist_single_batch = next(iter(data_loader))
  img_array = np.array(mnist_single_batch[0]*255).squeeze(1)
  cv2.imwrite("mninst_%s.jpg"%i,img_array[i])

In [61]:
# CNN Model (2-layers)
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = torch.nn.Sequential(
        # 1개의 채널을 입력받아 32채널로 컨벌루션
        torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        # 활성함수 ReLU
        torch.nn.ReLU(),
        # MaXpooling을 통해 이미지의 사이즈를 줄임
        # 작은크기의 사진으로도 사진을 구분할 수 있는 것과 같은 이치
        torch.nn.MaxPool2d(kernel_size=2, stride=2))
    
    self.layer2 = torch.nn.Sequential(
        # 32개의 채널을 입력받아 64채널로 컨벌루션
        # 64개의 피쳐맵을 형성한다
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        # 활성함수
        torch.nn.ReLU(),
        # 맥스풀링
        torch.nn.MaxPool2d(kernel_size=2, stride=2))
    
    # 뉴런간 선형 결합
    self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
    torch.nn.init.xavier_uniform_(self.fc.weight)

# 포워드 함수가 호출되는 시점
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1)
    out = self.fc(out)
    return out

In [62]:
# CNN모델이 gpu를 사용하도록 만들기
model = CNN().to(device)

In [79]:
img,label = next(iter(data_loader))
img = img.to(device)
label = label.to(device)
out = model.layer1(img)
print(out.shape)
out = model.layer2(out)
print(out.shape)
out = out.view(out.size(0), -1)
print(out.shape)
out = model.fc(out)

print(out[0])
print(label)

# soft argmax
print(torch.argmax(out, 1))

#이 손실함수를 통해서 엔트로피를 낮추는 방식으로 진행됨
loss = torch.nn.CrossEntropyLoss()
output = loss(out, label)
print(output)




torch.Size([100, 32, 14, 14])
torch.Size([100, 64, 7, 7])
torch.Size([100, 3136])
Parameter containing:
tensor([[-0.0093,  0.0033,  0.0044,  ...,  0.0407, -0.0290,  0.0295],
        [-0.0164,  0.0260,  0.0367,  ...,  0.0041, -0.0019,  0.0264],
        [ 0.0306, -0.0012,  0.0436,  ...,  0.0148,  0.0073,  0.0019],
        ...,
        [ 0.0017, -0.0417, -0.0415,  ..., -0.0223, -0.0142,  0.0004],
        [-0.0380,  0.0069,  0.0016,  ..., -0.0204,  0.0266,  0.0097],
        [ 0.0398,  0.0293,  0.0321,  ..., -0.0234, -0.0282, -0.0056]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.0433,  0.0359, -0.0234,  ..., -0.0206,  0.0096,  0.0135],
        [-0.0322, -0.0363, -0.0419,  ..., -0.0350, -0.0059, -0.0166],
        [ 0.0093, -0.0352, -0.0370,  ..., -0.0049,  0.0157, -0.0366],
        ...,
        [-0.0187, -0.0003, -0.0201,  ...,  0.0231,  0.0076, -0.0136],
        [-0.0041, -0.0215,  0.0325,  ..., -0.0007, -0.0242,  0.0344],
        [ 0.0432,  0.0029,  0.016

In [63]:
# 손실함수
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [64]:
total_batch = len(data_loader)

for epoch in range(15):

  # loss 값을 담을 변수 0으로 초기화
  avg_cost = 0

  # img는 input, label은 정답지
  for img,label in data_loader:
    img = img.to(device)
    label = label.to(device)

    # 선형회귀에서 했던 코드
    optimizer.zero_grad()
    hypothesis = model(img)
    cost = criterion(hypothesis, label)

    cost.backward()
    optimizer.step()

    avg_cost += cost/total_batch

  print("Epoch:{} cost={}".format(epoch+1, avg_cost))

Epoch:1 cost=0.21711450815200806
Epoch:2 cost=0.0627535954117775
Epoch:3 cost=0.046515125781297684
Epoch:4 cost=0.037708017975091934
Epoch:5 cost=0.03144245222210884
Epoch:6 cost=0.026822924613952637
Epoch:7 cost=0.022707849740982056
Epoch:8 cost=0.019798707216978073
Epoch:9 cost=0.015502367168664932
Epoch:10 cost=0.014321819879114628
Epoch:11 cost=0.012325611896812916
Epoch:12 cost=0.010221793316304684
Epoch:13 cost=0.010157003998756409
Epoch:14 cost=0.007247631903737783
Epoch:15 cost=0.007243932690471411


In [68]:
with torch.no_grad():

  img_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  img_test.shape
  prediction = model(img_test)


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


In [77]:
print(prediction)
print(torch.argmax(prediction,axis=1))

tensor([[-7716.6250, -1909.7129, -1820.0050,  ...,  5691.4263, -3788.3672,
           -70.1264],
        [-2570.8496, -1903.1008,  4586.4033,  ..., -5711.3135, -2791.0190,
         -4471.4292],
        [-4939.8022,  2988.5381, -2379.4253,  ...,  -764.2632,  -507.3808,
         -2871.1953],
        ...,
        [-8060.3398, -3253.9475, -6693.4263,  ...,  -633.6251, -1276.9800,
         -1904.2771],
        [-1680.8075, -7005.0889, -5313.4365,  ..., -4458.7275,  2067.0857,
         -6992.3237],
        [-4085.7605, -7682.7339, -2169.1772,  ..., -7115.5938, -2194.2795,
         -6560.6899]], device='cuda:0')
tensor([7, 2, 1,  ..., 4, 5, 6], device='cuda:0')


In [ ]:
# 정확도 테스트
with torch.no_grad():

    # 테스트 데이터 불러와서 1렬 뉴런 형태로 불러오기
    img_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    label_test = mnist_test.test_labels.to(device)
    
    # 학습된 CNN모델에 삽입
    prediction = model(img_test)
    # 테스트 라벨과 prediction의 라벨을 비교해서 값이 True인것만 추출
    correct_prediction = torch.argmax(prediction, 1) == label_test

    # 모은 데이터의 평균을 내서 정확도를 추출
    accuracy = correct_prediction.float().mean()

    # 정확도 프린트
    print('Accuracy:', accuracy.item())

Accuracy: 0.9886999726295471


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
